In [1]:
import netCDF4
import itertools

from tqdm import tqdm

Nx = 512
Ny = 512
Nz = 601
Na = 3
#Ns = Nx * Ny
Ns = 16

In [9]:
#mu_id = ['10', '8', '6', '4', '2']
mu_id = ['10', '6', '2']

T = np.zeros((Na, Nz, Ny * Nx))
t = np.zeros((Na, Nz, Ny * Nx))

T_ssd = np.zeros((Na, Nz, Ns))
t_ssd = np.zeros((Na, Nz, Ns))

T_300 = np.zeros((Na, Nz, Ns))
t_300 = np.zeros((Na, Nz, Ns))

for i in tqdm(range(Na)):
    
    t[i, :, :] = np.array(netCDF4.Dataset('./ssd/222809/' + mu_id[i] + '/taugrid.222809.nc.1')['tau']).reshape(Nz, Ny * Nx)
    T[i, :, :] = np.array(netCDF4.Dataset('./ssd/222809/' + mu_id[i] + '/T_onTau.222809.nc.1')['T']).reshape(Nz, Ny * Nx)
    
    sample = np.random.choice(Nx * Ny, Ns, replace = False)
    
    t_ssd[i, :, :] = t[i, :, sample].T
    T_ssd[i, :, :] = T[i, :, sample].T

    t[i, :, :] = np.array(netCDF4.Dataset('./300G/627321/' + mu_id[i] + '/taugrid.627321.nc.1')['tau']).reshape(Nz, Ny * Nx)
    T[i, :, :] = np.array(netCDF4.Dataset('./300G/627321/' + mu_id[i] + '/T_onTau.627321.nc.1')['T']).reshape(Nz, Ny * Nx)
    
    sample = np.random.choice(Nx * Ny, Ns, replace = False)
    
    t_300[i, :, :] = t[i, :, sample].T
    T_300[i, :, :] = T[i, :, sample].T

np.savez('intro_img_2_sample_16', t_ssd = t_ssd, t_300 = t_300, T_ssd = T_ssd, T_300 = T_300)

  0%|                                                                                                                                                   | 0/3 [00:08<?, ?it/s]


KeyboardInterrupt: 

In [10]:
print('reading t_ssd...')
t_ssd = np.load('intro_img_2.npz')['t_ssd']
print('reading t_300...')
t_300 = np.load('intro_img_2.npz')['t_300']
print('reading T_ssd...')
T_ssd = np.load('intro_img_2.npz')['T_ssd']
print('reading T_ssd...')
T_300 = np.load('intro_img_2.npz')['T_300']

reading t_ssd...
reading t_300...
reading T_ssd...
reading T_ssd...


In [2]:
print('reading t_ssd...')
t_ssd = np.load('intro_img_2_sample_16.npz')['t_ssd']
print('reading t_300...')
t_300 = np.load('intro_img_2_sample_16.npz')['t_300']
print('reading T_ssd...')
T_ssd = np.load('intro_img_2_sample_16.npz')['T_ssd']
print('reading T_ssd...')
T_300 = np.load('intro_img_2_sample_16.npz')['T_300']

reading t_ssd...
reading t_300...
reading T_ssd...
reading T_ssd...


In [5]:
import itertools

from scipy import interpolate

import sys

def find_interpol_range(tau, Nz, Ns):

    thick = 1e+20
    thinn = 1e-20

    for k in range(Ns):

        opt_thick_end = tau[Nz - 1, k]
        opt_thinn_end = tau[0, k]

        if opt_thinn_end > thinn: thinn = opt_thinn_end
        
        if opt_thick_end < thick: thick = opt_thick_end
    
    return np.log10(thinn), np.log10(thick)

def find_shortest_delta(tau, Nz, Ns):
    
    tau_1d = np.log10(tau.T).reshape(Nz * Ns)
    
    delta = np.abs(np.diff(tau_1d))
    
    idx_min = np.argmin(np.abs(delta - 0.001))
    
#    delta = 20
    
#    for j, k in tqdm(itertools.product(range(Nz - 1), range(Ns)), desc = str(i) + ', ' + cube + ', find shortest delta'):
        
#        d = np.abs(np.log10(tau[j + 1, k]) - np.log10(tau[j, k]))
        
#        if d < delta and d > 0.001: delta = d

#    print(delta[idx_min])

    #print(np.min(np.abs(delta - 0.001)))
    
#    sys.exit()
            
    return delta[idx_min]

def const_interpol_grid(thin, thick, delta):
    
    Ng = int(np.floor(np.abs(thick - thin) / delta))
    
    grid = np.zeros(Ng)
    
    grid[0] = thin
    
    for i in range(Ng - 1): grid[i + 1] = grid[i] + delta
    
    return grid

def interpol_and_average(i, cube, tau, T, igrid, Ns, up, lp):
    
    Ti = np.zeros((Ns, len(igrid)), dtype = np.float32)
    
    T_mean = np.zeros(len(igrid))
    
    T_up = np.zeros((len(up), len(igrid)))
    T_lp = np.zeros((len(lp), len(igrid)))
    
    for k in tqdm(range(Ns), desc = str(i) + ', ' + cube + ', interpolating'):
    
        f = interpolate.interp1d(np.log10(tau[:, k]), T[:, k])
        
        Ti[k, :] = f(igrid)

    print(str(i) + ', ' + cube + ', averaging...')

    T_mean = np.mean(Ti, axis = 0)
    
    for m, j in tqdm(itertools.product(range(len(up)), range(len(igrid))), desc = str(i) + ', ' + cube + ', calculating percentiles'):

        T_lp[m, j] = np.percentile(Ti[:, j], lp[m])
        T_up[m, j] = np.percentile(Ti[:, j], up[m])

    return T_lp, T_mean, T_up

lp = np.array([0,   1,  5,  10, 25])
up = np.array([100, 99, 95, 90, 75])

grid_ssd = np.ones((Na, 110000)) * np.nan
grid_300 = np.ones((Na, 110000)) * np.nan

T_mean_ssd = np.ones((Na, 110000)) * np.nan
T_mean_300 = np.ones((Na, 110000)) * np.nan

T_lp_ssd = np.ones((Na, len(lp), 110000)) * np.nan
T_lp_300 = np.ones((Na, len(lp), 110000)) * np.nan

T_up_ssd = np.ones((Na, len(up), 110000)) * np.nan
T_up_300 = np.ones((Na, len(up), 110000)) * np.nan

In [ ]:
for i in range(Na):
    
    print('SSD:')
    
    print('finding interpolation range...')
    thin, thick =        find_interpol_range(t_ssd[i, :, :], Nz, Ns)
    
    print('finding shortest delta...')
    delta =              find_shortest_delta(t_ssd[i, :, :], Nz, Ns)
    
    print('constructing interpolation grid...')
    grid =               const_interpol_grid(thin, thick, delta)
    
    T_lp, T_mean, T_up = interpol_and_average(i, 'ssd', t_ssd[i, :, :], T_ssd[i, :, :], grid, Ns, up, lp)
    
    grid_ssd[i, : len(grid)] = 10.0**grid
    
    T_mean_ssd[i, : len(T_mean)] = T_mean
    
    T_lp_ssd[i, :, : len(T_mean)] = T_lp
    T_up_ssd[i, :, : len(T_mean)] = T_up
    
    print('300G:')
    
    print('finding interpolation range...')
    thin, thick =        find_interpol_range(t_300[i, :, :], Nz, Ns)
    
    print('finding shortest delta...')
    delta =              find_shortest_delta(t_300[i, :, :], Nz, Ns)
    
    print('constructing interpolation grid...')
    grid =               const_interpol_grid(thin, thick, delta)
    
    T_lp, T_mean, T_up = interpol_and_average(i, '300', t_300[i, :, :], T_300[i, :, :], grid, Ns, up, lp)
    
    grid_300[i, : len(grid)] = 10.0**grid
    
    T_mean_300[i, : len(T_mean)] = T_mean
    
    T_lp_300[i, :, : len(T_mean)] = T_lp
    T_up_300[i, :, : len(T_mean)] = T_up

np.savez('intro_img_interpol', grid_ssd = grid_ssd, \
                               grid_300 = grid_300, \
                               T_mean_ssd = T_mean_ssd, \
                               T_mean_300 = T_mean_300, \
                               T_lp_ssd = T_lp_ssd, \
                               T_lp_300 = T_lp_300, \
                               T_up_ssd = T_up_ssd, \
                               T_up_300 = T_up_300)

In [10]:
print('reading grid_ssd...')
grid_ssd =   np.load('intro_img_interpol.npz')['grid_ssd']
print('reading T_mean_ssd...')
T_lp_ssd =   np.load('intro_img_interpol.npz')['T_lp_ssd']
T_mean_ssd = np.load('intro_img_interpol.npz')['T_mean_ssd']
T_up_ssd =   np.load('intro_img_interpol.npz')['T_up_ssd']
print('reading grid_300...')
grid_300 =   np.load('intro_img_interpol.npz')['grid_300']
print('reading T_mean_300...')
T_lp_300 =   np.load('intro_img_interpol.npz')['T_lp_300']
T_mean_300 = np.load('intro_img_interpol.npz')['T_mean_300']
T_up_300 =   np.load('intro_img_interpol.npz')['T_up_300']

reading grid_ssd...
reading T_mean_ssd...
reading grid_300...
reading T_mean_300...


In [3]:
print('reading grid_ssd...')
grid_ssd =        np.load('intro_img_interpol_p0_100.npz')['grid_ssd']
print('reading T_mean_ssd...')
T_lp_ssd_p0_100 = np.load('intro_img_interpol_p0_100.npz')['T_lp_ssd']
T_mean_ssd =      np.load('intro_img_interpol_p0_100.npz')['T_mean_ssd']
T_up_ssd_p0_100 = np.load('intro_img_interpol_p0_100.npz')['T_up_ssd']
print('reading grid_300...')
grid_300 =        np.load('intro_img_interpol_p0_100.npz')['grid_300']
print('reading T_mean_300...')
T_lp_300_p0_100 = np.load('intro_img_interpol_p0_100.npz')['T_lp_300']
T_mean_300 =      np.load('intro_img_interpol_p0_100.npz')['T_mean_300']
T_up_300_p0_100 = np.load('intro_img_interpol_p0_100.npz')['T_up_300']

reading grid_ssd...
reading T_mean_ssd...
reading grid_300...
reading T_mean_300...


In [4]:
print('reading grid_ssd...')
grid_ssd =       np.load('intro_img_interpol_p5_95.npz')['grid_ssd']
print('reading T_mean_ssd...')
T_lp_ssd_p5_95 = np.load('intro_img_interpol_p5_95.npz')['T_lp_ssd']
T_mean_ssd =     np.load('intro_img_interpol_p5_95.npz')['T_mean_ssd']
T_up_ssd_p5_95 = np.load('intro_img_interpol_p5_95.npz')['T_up_ssd']
print('reading grid_300...')
grid_300 =       np.load('intro_img_interpol_p5_95.npz')['grid_300']
print('reading T_mean_300...')
T_lp_300_p5_95 = np.load('intro_img_interpol_p5_95.npz')['T_lp_300']
T_mean_300 =     np.load('intro_img_interpol_p5_95.npz')['T_mean_300']
T_up_300_p5_95 = np.load('intro_img_interpol_p5_95.npz')['T_up_300']

reading grid_ssd...
reading T_mean_ssd...
reading grid_300...
reading T_mean_300...


In [23]:
import matplotlib.pyplot as plt

import matplotlib.gridspec as gridspec

from tqdm import tqdm

from matplotlib.ticker import LogLocator
from matplotlib.ticker import MultipleLocator
from matplotlib.ticker import AutoMinorLocator

props = dict(boxstyle = 'round', facecolor = 'wheat', alpha = 0.5)

plt.rcParams.update({'font.size': 8})
plt.rcParams["font.family"] = 'Times New Roman'

mus = ['1.0', '0.8', '0.6', '0.4', '0.2']

fig = plt.figure(dpi = 300)

fig.set_size_inches(10, 11.25, forward = True)

fig.tight_layout()

gs = gridspec.GridSpec(5, 3, width_ratios = [1, 1, 1], hspace = 0.2, wspace = 0.2)

for i in tqdm(range(Na)):

    ssd = plt.subplot(gs[i, 0])
    mag = plt.subplot(gs[i, 1])
        
    com = plt.subplot(gs[i, 2])
    
#    for k in np.random.choice(Nx * Ny, Ns, replace = False):
    for k in range(Ns):
        
        ssd.plot(t_ssd[i, :, k], T_ssd[i, :, k], color = 'gray')
        mag.plot(t_300[i, :, k], T_300[i, :, k], color = 'gray')
        
    ssd.plot(grid_ssd[i, :], Ti_ssd[i, :], color = 'k')
    mag.plot(grid_300[i, :], Ti_300[i, :], color = 'r')
    
    com.plot(grid_ssd[i, :], Ti_ssd[i, :], color = 'k', label = 'SSD')
    com.plot(grid_300[i, :], Ti_300[i, :], color = 'r', label = '300G')
        
#        if i == 0:
            
#            ssd.axvline(x = thick, color = 'r', linestyle = '--')
#            ssd.axvline(x = thinn, color = 'r', linestyle = '--')
        
    ssd.set_xscale('log')
    
    ssd.set_xlim(3e+1, 2e-12)
    ssd.set_ylim(2000, 10500)
    
    ssd.set_ylabel('Temperature, K')
    
    if i == 0: ssd.set_title('SSD')
    
    mag.set_xscale('log')
    
    mag.set_xlim(ssd.get_xlim())
    mag.set_ylim(ssd.get_ylim())
    
    if i == 0: mag.set_title('300G')
        
    com.set_xscale('log')
    
    com.set_xlim(3e+1, 2e-12)
    com.set_ylim(2000, 10500)
    
    if i == 4: ssd.set_xlabel(r'$\tau_\mathrm{Ross}$')
    if i == 4: mag.set_xlabel(r'$\tau_\mathrm{Ross}$')
    if i == 4: com.set_xlabel(r'$\tau_\mathrm{Ross}$')
    
    com.set_ylabel(r'$\mu =$' + mus[i])
    com.yaxis.set_label_position("right")
    
    if i == 0: com.legend(framealpha = 1, loc = 1, handletextpad = 1, prop = {'size': 7.5})

plt.savefig('intro_img_2.pdf', bbox_inches = 'tight')

plt.close('all')

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00,  9.27it/s]


In [69]:
import matplotlib.pyplot as plt

import matplotlib.gridspec as gridspec

from tqdm import tqdm

from matplotlib.ticker import LogLocator
from matplotlib.ticker import MultipleLocator
from matplotlib.ticker import AutoMinorLocator

props = dict(boxstyle = 'round', facecolor = 'wheat', alpha = 0.5)

plt.rcParams.update({'font.size': 8})
plt.rcParams["font.family"] = 'Times New Roman'

mus = ['1.0', '0.6', '0.2']

fig = plt.figure(dpi = 300)

fig.set_size_inches(10, 11.25 * 3 / 5, forward = True)

fig.tight_layout()

gs = gridspec.GridSpec(3, 3, width_ratios = [1, 1, 1], hspace = 0.2, wspace = 0.2)

#idx_mu = [0, 2, 4]

for i in tqdm(range(Na)):

    ssd = plt.subplot(gs[i, 0])
    mag = plt.subplot(gs[i, 1])
        
    com = plt.subplot(gs[i, 2])
    
#    for k in np.random.choice(Nx * Ny, Ns, replace = False):
    for k in range(Ns):
        
        if k == 0:

            ssd.plot(t_ssd[i, :, k], T_ssd[i, :, k], color = 'k', linewidth = 0.5, label = 'individual')
            mag.plot(t_300[i, :, k], T_300[i, :, k], color = 'k', linewidth = 0.5, label = 'individual')

        else:

            ssd.plot(t_ssd[i, :, k], T_ssd[i, :, k], color = 'k', linewidth = 0.5)
            mag.plot(t_300[i, :, k], T_300[i, :, k], color = 'k', linewidth = 0.5)

    ssd.plot(grid_ssd[i, :], T_mean_ssd[i, :], color = 'blue', label = 'average')
    ssd.fill_between(grid_ssd[i, :], T_lp_ssd_p5_95[i, :],  T_up_ssd_p5_95[i, :],  color = 'green', alpha = 0.50, label = '5% to 95 %')
    ssd.fill_between(grid_ssd[i, :], T_lp_ssd_p0_100[i, :], T_up_ssd_p0_100[i, :], color = 'green', alpha = 0.25, label = 'min to max')
    
    mag.plot(grid_300[i, :], T_mean_300[i, :], color = 'orange', label = 'average')
    mag.fill_between(grid_300[i, :], T_lp_300_p5_95[i, :],  T_up_300_p5_95[i, :],  color = 'purple', alpha = 0.50, label = '5% to 95 %')
    mag.fill_between(grid_300[i, :], T_lp_300_p0_100[i, :], T_up_300_p0_100[i, :], color = 'purple', alpha = 0.25, label = 'min to max')
    
    com.plot(grid_ssd[i, :], T_mean_ssd[i, :], color = 'blue',   label = 'SSD')
    com.plot(grid_300[i, :], T_mean_300[i, :], color = 'orange', label = '300G')
    
    com.fill_between(grid_ssd[i, :], T_lp_ssd_p5_95[i, :], T_up_ssd_p5_95[i, :], color = 'green',  alpha = 0.5, label = 'SSD')
    com.fill_between(grid_300[i, :], T_lp_300_p5_95[i, :], T_up_300_p5_95[i, :], color = 'purple', alpha = 0.5, label = '300G')
        
    ssd.set_xscale('log')
    
    ssd.set_xlim(3e+1, 2e-12)
    ssd.set_ylim(2000, 10500)
    
    ssd.yaxis.set_minor_locator(AutoMinorLocator(4))
    
    ssd.set_ylabel('Temperature, K')
    
    if i == 0:
        
        ssd.set_title('SSD')
        
        leg0 = ssd.legend(framealpha = 1, loc = 1, handletextpad = 1, prop = {'size': 7.5})
        leg1 = mag.legend(framealpha = 1, loc = 1, handletextpad = 1, prop = {'size': 7.5})
        leg2 = com.legend(framealpha = 1, loc = 1, handletextpad = 1, prop = {'size': 10.0})
    
    mag.set_xscale('log')
    
    mag.set_xlim(ssd.get_xlim())
    mag.set_ylim(ssd.get_ylim())
    
    mag.yaxis.set_minor_locator(AutoMinorLocator(4))
    
    if i == 0: mag.set_title('300G')
        
    com.set_xscale('log')
    
    #com.set_xlim(ssd.get_xlim())
    com.set_xlim(3e+1, 1e-9)
    com.set_ylim(ssd.get_ylim())
    
    com.yaxis.set_minor_locator(AutoMinorLocator(4))
    
    if i == 2: ssd.set_xlabel(r'$\tau_\mathrm{Ross}$')
    if i == 2: mag.set_xlabel(r'$\tau_\mathrm{Ross}$')
    if i == 2: com.set_xlabel(r'$\tau_\mathrm{Ross}$')
    
    com.set_ylabel(r'$\mu =$' + mus[i])
    com.yaxis.set_label_position("right")
    
    if i == 0: com.legend(framealpha = 1, loc = 1, handletextpad = 1, prop = {'size': 7.5})

plt.savefig('intro_img_2.pdf', bbox_inches = 'tight')

plt.close('all')

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  7.26it/s]


In [9]:
import itertools

for i, k, j in itertools.product(range(Na), range(Ns), range(Nz - 1)):
            
    delta = np.abs(np.log10(t_ssd[i, j, k]) - np.log10(t_ssd[i, j + 1, k]))
        
    if np.abs(delta - 0.0001) < 1e-6: T_ssd[i, j, k] = np.nan
    
    delta = np.abs(np.log10(t_300[i, j, k]) - np.log10(t_300[i, j + 1, k]))

    if np.abs(delta - 0.0001) < 1e-6: T_300[i, j, k] = np.nan
            
np.savez('intro_img_2_nans', t_ssd = t_ssd, t_300 = t_300, T_ssd = T_ssd, T_300 = T_300)
            
t_ssd = np.load('intro_img_2_nans.npz')['t_ssd']
t_300 = np.load('intro_img_2_nans.npz')['t_300']
T_ssd = np.load('intro_img_2_nans.npz')['T_ssd']
T_300 = np.load('intro_img_2_nans.npz')['T_300']

In [8]:
grid_ssd_t200 =   np.load('tau200_rot_interpol.npz')['grid_ssd']
grid_mag_t200 =   np.load('tau200_rot_interpol.npz')['grid_mag']

T_mean_ssd_t200 = np.load('tau200_rot_interpol.npz')['Ti_ssd']
T_mean_mag_t200 = np.load('tau200_rot_interpol.npz')['Ti_mag']

In [20]:
import matplotlib.pyplot as plt

import matplotlib.gridspec as gridspec

from tqdm import tqdm

from matplotlib.ticker import LogLocator
from matplotlib.ticker import MultipleLocator
from matplotlib.ticker import AutoMinorLocator

props = dict(boxstyle = 'round', facecolor = 'wheat', alpha = 0.5)

plt.rcParams.update({'font.size': 8})
plt.rcParams["font.family"] = 'Times New Roman'

mus = ['1.0', '0.6', '0.2']

fig = plt.figure(dpi = 300)

fig.set_size_inches(10, 11.25 * 3 / 5, forward = True)

fig.tight_layout()

gs = gridspec.GridSpec(3, 4, width_ratios = [1, 1, 1, 1], hspace = 0.2, wspace = 0.3)

#idx_mu = [0, 2, 4]

for i in tqdm(range(Na)):

    ssd = plt.subplot(gs[i, 0])
    mag = plt.subplot(gs[i, 1])
        
    com = plt.subplot(gs[i, 2])
    
    t20 = plt.subplot(gs[i, 3])
    
    for m in range(len(lp)):
        
        if m == 0: label = r'$p_0$ to $p_{100}$'; alpha = 0.1
        if m == 1: label = r'$p_1$ to $p_{99}$';  alpha = 0.2
        if m == 2: label = r'$p_5$ to $p_{95}$';  alpha = 0.3
        if m == 3: label = r'$p_{10}$ to $p_{90}$'; alpha = 0.4
        if m == 4: label = r'$p_{25}$ to $p_{75}$'; alpha = 1.0
        
        ssd.fill_between(grid_ssd[i, :], T_lp_ssd[i, m, :],  T_up_ssd[i, m, :], color = 'green',  alpha = alpha, label = label)
        mag.fill_between(grid_300[i, :], T_lp_300[i, m, :],  T_up_300[i, m, :], color = 'purple', alpha = alpha, label = label)
        
#    for k in np.random.choice(Nx * Ny, Ns, replace = False):
    for k in range(0, Ns, 2):
        
        if k == 0:

            ssd.plot(t_ssd[i, :, k], T_ssd[i, :, k], color = 'k', alpha = 0.25, linewidth = 0.5, label = 'individual')
            mag.plot(t_300[i, :, k], T_300[i, :, k], color = 'k', alpha = 0.25, linewidth = 0.5, label = 'individual')

        else:

            ssd.plot(t_ssd[i, :, k], T_ssd[i, :, k], color = 'k', alpha = 0.25, linewidth = 0.5)
            mag.plot(t_300[i, :, k], T_300[i, :, k], color = 'k', alpha = 0.25, linewidth = 0.5)
        
    ssd.plot(grid_ssd[i, :], T_mean_ssd[i, :], color = 'cyan',   label = 'average') 
    mag.plot(grid_300[i, :], T_mean_300[i, :], color = 'orange', label = 'average')
    
    com.plot(grid_ssd[i, :], T_mean_ssd[i, :], color = 'cyan',   label = 'SSD')
    com.plot(grid_300[i, :], T_mean_300[i, :], color = 'orange', label = '300G')
    
    com.fill_between(grid_ssd[i, :], T_lp_ssd[i, 2, :], T_up_ssd[i, 2, :], color = 'green',  alpha = 0.3, label = r'SSD: $p_5$ to $p_{95}$')
    com.fill_between(grid_300[i, :], T_lp_300[i, 2, :], T_up_300[i, 2, :], color = 'purple', alpha = 0.3, label = r'300G: $p_5$ to $p_{95}$')
    
    t20.plot(grid_ssd_t200[i, :], T_mean_ssd_t200[i, :], color = 'cyan',   label = 'SSD')
    t20.plot(grid_mag_t200[i, :], T_mean_mag_t200[i, :], color = 'orange', label = '300G')
        
    ssd.set_xscale('log')
    
    ssd.set_xlim(3e+1, 2e-12)
    ssd.set_ylim(2000, 10500)
    
    ssd.yaxis.set_minor_locator(AutoMinorLocator(4))
    
    ssd.set_ylabel('Temperature, K')
    
    mag.set_xscale('log')
    
    mag.set_xlim(ssd.get_xlim())
    mag.set_ylim(ssd.get_ylim())
    
    mag.yaxis.set_minor_locator(AutoMinorLocator(4))
        
    com.set_xscale('log')
    
    com.set_xlim(3e+1, 1e-9)
    com.set_ylim(ssd.get_ylim())
    
    com.yaxis.set_minor_locator(AutoMinorLocator(4))
    
    t20.set_ylabel(r'$\mu =$' + mus[i])
    t20.yaxis.set_label_position("right")
    
    t20.set_xscale('log')
    
    t20.set_xlim(4e+3, 1e-1)
    t20.set_ylim(2750, 5500)
    
    if i == 0:
        
        ssd.set_title('SSD')
        mag.set_title('300G')
        
        ssd.legend(framealpha = 1, loc = 1, handletextpad = 1, prop = {'size': 6.5})
        mag.legend(framealpha = 1, loc = 1, handletextpad = 1, prop = {'size': 6.5})
        com.legend(framealpha = 1, loc = 1, handletextpad = 1, prop = {'size': 6.5})
        
    if i == 2: ssd.set_xlabel(r'$\tau_\mathrm{Ross}$')
    if i == 2: mag.set_xlabel(r'$\tau_\mathrm{Ross}$')
    if i == 2: com.set_xlabel(r'$\tau_\mathrm{Ross}$')
    if i == 2: t20.set_xlabel(r'$\tau_{200}$')

plt.savefig('temp_struct.pdf', bbox_inches = 'tight')

plt.close('all')

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  6.31it/s]
